In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import pandas_datareader as pdr
import os
import talib as tl
from sklearn.preprocessing import LabelBinarizer, StandardScaler
from sklearn_pandas import DataFrameMapper
%matplotlib inline

In [2]:
def fills(df):
    df.fillna(method='ffill', inplace=True)
    df.fillna(method='bfill', inplace=True)
    return df

In [3]:
aapl = fills(pdr.get_data_google('AAPL', '2000-01-01', '2018-01-01'))
spy = fills(pdr.get_data_google('SPY', '2000-01-01', '2018-01-01'))

In [4]:
oopen = aapl['Open'].values
high = aapl['High'].values
low = aapl['Low'].values
close = aapl['Close'].values

In [5]:
runup252 = aapl['Close'].pct_change(252)
aapl['Run Up 252'] = runup252

In [6]:
beta63 = tl.BETA(close, spy['Close'].values, timeperiod=63)
aapl['Beta 63'] = beta63

In [7]:
ema100 = tl.EMA(close, timeperiod=100)
aapl['EMA 100'] = ema100
aapl['EMA 100'] = aapl['EMA 100'].pct_change()

In [8]:
sma100 = tl.SMA(close, timeperiod=100)
aapl['SMA 100'] = sma100
aapl['SMA 100'] = aapl['SMA 100'].pct_change()

In [9]:
sma_mom100 = aapl['SMA 100'] - aapl['SMA 100'].shift(1)
aapl['SMA MOM 100'] = sma_mom100

In [10]:
sp500_sma100 = tl.SMA(spy['Close'].values, timeperiod=100)
aapl['SP500 SMA 100'] = sp500_sma100
aapl['SP500 SMA 100'] = aapl['SP500 SMA 100'].pct_change()

In [11]:
sp500vola = tl.ATR(spy['High'].values, spy['Low'].values, spy['Close'].values, timeperiod=63)
aapl['SP500 Vola 63'] = sp500vola

In [12]:
sharpe_days = 63
daily_ret = aapl['Close'].pct_change()
sharpe63 = np.sqrt(sharpe_days)*(daily_ret.rolling(sharpe_days).mean()/daily_ret.rolling(sharpe_days).std())
aapl['Sharpe 63'] = sharpe63

In [13]:
vola63 = tl.ATR(high, low, close, timeperiod=63)
aapl['Vola 63'] = vola63

In [14]:
return_days = 10
extreme_return = 5
ret = 100*aapl['Close'].pct_change(return_days).shift(-return_days)
aapl[str(return_days) + ' days future return'] = ret

In [15]:
aapl['Weekday'] = aapl.index.dayofweek.map(lambda x: {0: 'Mon', 1: 'Tue', 2: 'Wed', 3: 'Thu', 4: 'Fri'}.get(x))

In [16]:
def to_class(val, extreme):
    if val < -extreme:
        return 'Neg'
    elif val > extreme:
        return 'Pos'
    else:
        return 'Neutral'
aapl['Return class'] = aapl[str(return_days) + ' days future return'].apply(lambda val: to_class(val, extreme_return))
aapl['Return class'].head()

Date
2002-02-11        Neg
2002-02-12        Neg
2002-02-13        Neg
2002-02-14    Neutral
2002-02-15    Neutral
Name: Return class, dtype: object

In [17]:
aapl.tail(6)

,Open,High,Low,Close,Volume,Run Up 252,Beta 63,EMA 100,SMA 100,SMA MOM 100,SP500 SMA 100,SP500 Vola 63,Sharpe 63,Vola 63,10 days future return,Weekday,Return class
Date,,,,,,,,,,,,,,,,,
2017-12-21,174.17,176.02,174.10,175.01,20949896,0.495045,0.141154,0.001409,0.001530,-0.000043,0.000797,1.454152,1.690945,2.485687,NaN,Thu,Neutral
2017-12-22,174.68,175.42,174.50,175.01,16349444,0.504945,0.139007,0.001379,0.001094,-0.000436,0.000789,1.442816,1.808430,2.460835,NaN,Fri,Neutral
2017-12-26,170.80,171.47,169.68,170.57,33185536,0.463869,0.138846,0.000813,0.000917,-0.000177,0.000795,1.429915,1.263734,2.506377,NaN,Tue,Neutral
2017-12-27,170.10,170.78,169.71,170.60,21498213,0.454887,0.137480,0.000800,0.000868,-0.000049,0.000782,1.418646,1.187278,2.483577,NaN,Wed,Neutral
2017-12-28,171.00,171.85,170.48,171.08,16480187,0.465228,0.138857,0.000841,0.000749,-0.000119,0.000785,1.405652,1.294132,2.465901,NaN,Thu,Neutral
2017-12-29,170.52,170.59,169.22,169.23,25999922,0.449756,0.142824,0.000601,0.000558,-0.000191,0.000768,1.413657,1.098579,2.456284,NaN,Fri,Neutral


In [18]:
aapl.drop(['Open', 'High', 'Low', 'Close', 'Volume', str(return_days) + ' days future return'], axis=1, inplace=True)
aapl.dropna(inplace=True)

### Resample umbalanced data and Train-test split

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.utils import resample

In [20]:
train_index = int(len(aapl) * 0.75)
aapl_train = aapl.iloc[:train_index]
aapl_test = aapl.iloc[train_index:]

In [21]:
df_majority = aapl_train[aapl_train['Return class']=='Pos']
df_minority = aapl_train[aapl_train['Return class']=='Neg']
df_other = aapl_train[aapl_train['Return class']=='Neutral']

df_minority_upsampled = resample(df_minority, 
                                 replace=True,
                                 n_samples=len(df_majority))

aapl_train = pd.concat([df_majority, df_minority_upsampled, df_other])
 
aapl_train['Return class'].value_counts()

Neutral    1477
Neg         900
Pos         900
Name: Return class, dtype: int64

In [22]:
X_train = aapl_train.drop('Return class', axis=1)
X_test = aapl_test.drop('Return class', axis=1)
y_train = aapl_train['Return class']
y_test = aapl_test['Return class']

In [23]:
trafos = [([col], StandardScaler()) for col in aapl.columns[:-2].tolist()]
trafos = [('Weekday', LabelBinarizer())] + trafos
#trafos += [('Return class', LabelBinarizer())]

In [24]:
trafos

[('Weekday', LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)),
 (['Run Up 252'], StandardScaler(copy=True, with_mean=True, with_std=True)),
 (['Beta 63'], StandardScaler(copy=True, with_mean=True, with_std=True)),
 (['EMA 100'], StandardScaler(copy=True, with_mean=True, with_std=True)),
 (['SMA 100'], StandardScaler(copy=True, with_mean=True, with_std=True)),
 (['SMA MOM 100'], StandardScaler(copy=True, with_mean=True, with_std=True)),
 (['SP500 SMA 100'], StandardScaler(copy=True, with_mean=True, with_std=True)),
 (['SP500 Vola 63'], StandardScaler(copy=True, with_mean=True, with_std=True)),
 (['Sharpe 63'], StandardScaler(copy=True, with_mean=True, with_std=True)),
 (['Vola 63'], StandardScaler(copy=True, with_mean=True, with_std=True))]

In [25]:
mapper = DataFrameMapper(trafos, df_out=True, default=None)

In [26]:
X_train = mapper.fit_transform(X_train)
X_test = mapper.transform(X_test)
X_train.head()

,Weekday_Fri,Weekday_Mon,Weekday_Thu,Weekday_Tue,Weekday_Wed,Run Up 252,Beta 63,EMA 100,SMA 100,SMA MOM 100,SP500 SMA 100,SP500 Vola 63,Sharpe 63,Vola 63
Date,,,,,,,,,,,,,,
2003-04-11,1.0,0.0,0.0,0.0,0.0,-1.672967,-1.197562,-1.414049,-1.255014,-1.959955,-0.547869,0.304282,-1.227848,-1.220869
2003-04-15,0.0,0.0,0.0,1.0,0.0,-1.678483,-1.244461,-1.243625,-1.008104,0.545166,-0.373836,0.299802,-1.042360,-1.221293
2003-04-16,0.0,0.0,0.0,0.0,1.0,-1.696305,-1.261481,-1.303307,-1.115648,-0.850594,-0.275859,0.283765,-1.046736,-1.220696
2003-04-17,0.0,0.0,1.0,0.0,0.0,-1.687009,-1.188162,-1.362169,-1.366499,-1.974284,-0.641083,0.290914,-1.160033,-1.220643
2003-04-21,0.0,1.0,0.0,0.0,0.0,-1.668181,-1.236549,-1.347784,-1.260943,0.820359,-0.683601,0.279006,-1.003588,-1.221393


In [27]:
X_test.head()

,Weekday_Fri,Weekday_Mon,Weekday_Thu,Weekday_Tue,Weekday_Wed,Run Up 252,Beta 63,EMA 100,SMA 100,SMA MOM 100,SP500 SMA 100,SP500 Vola 63,Sharpe 63,Vola 63
Date,,,,,,,,,,,,,,
2014-04-14,0.0,1.0,0.0,0.0,0.0,-0.509175,0.947928,-0.644095,-0.523799,0.453058,-0.041434,-0.060189,-0.832390,0.823886
2014-04-15,0.0,0.0,0.0,1.0,0.0,-0.553684,0.820266,-0.696738,-0.556310,-0.262250,0.040356,-0.034369,-1.057906,0.829976
2014-04-16,0.0,0.0,0.0,0.0,1.0,-0.434695,0.810885,-0.678765,-0.517491,0.297071,0.163233,-0.030894,-1.214329,0.822872
2014-04-17,0.0,0.0,1.0,0.0,0.0,-0.352983,0.803767,-0.590473,-0.518949,-0.018749,0.105094,-0.042181,-1.070549,0.822830
2014-04-21,0.0,1.0,0.0,0.0,0.0,-0.318541,0.801193,-0.499845,-0.466551,0.403537,0.092596,-0.064755,-0.770756,0.820651


# ML model

In [28]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.grid_search import GridSearchCV

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

C:\DEV\Anaconda3\envs\trading\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\DEV\Anaconda3\envs\trading\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [29]:
def fit_and_print(grid, X_test, y_test, proba_min=None):
    grid.fit(X_train, y_train)
    best_estimator = grid.best_estimator_
    print('Best estimator:\n', str(best_estimator))
    
    if proba_min:
        proba = best_estimator.predict_proba(X_test)
        relevant = (proba[:,0] > proba_min) | (proba[:,2] > proba_min)
        X_test = X_test[relevant]
        y_test = y_test[relevant]
        
    y_pred = best_estimator.predict(X_test)
    
    print('Accuracy score:', str(accuracy_score(y_test, y_pred)))
    print('Classification report\n', classification_report(y_test, y_pred))
    print('Confusion matrix\n', str(confusion_matrix(y_test, y_pred)))

### K nearest

In [30]:
params = {'n_neighbors': np.arange(5,20,5),
          'metric': ['euclidean', 'manhattan', 'cityblock'],
          'leaf_size': np.arange(5,20,5), 
          'algorithm' : ['auto']}
grid = GridSearchCV(estimator=KNeighborsClassifier(n_jobs=4), param_grid=params, scoring='f1_weighted')

In [31]:
fit_and_print(grid, X_test, y_test)

Best estimator:
 KNeighborsClassifier(algorithm='auto', leaf_size=5, metric='euclidean',
           metric_params=None, n_jobs=4, n_neighbors=5, p=2,
           weights='uniform')
Accuracy score: 0.4386339381
Classification report
              precision    recall  f1-score   support

        Neg       0.09      0.47      0.16        77
    Neutral       0.77      0.51      0.62       707
        Pos       0.13      0.07      0.09       153

avg / total       0.61      0.44      0.49       937

Confusion matrix
 [[ 36  27  14]
 [284 364  59]
 [ 62  80  11]]


### Random Forest

In [32]:
params = {'n_estimators': np.arange(5,21,5),
          'max_features': np.arange(2,7),
          'min_samples_split': [2,3]}
grid = GridSearchCV(estimator=RandomForestClassifier(n_jobs=6, class_weight='balanced_subsample'), param_grid=params, scoring='f1_micro')
# grid = GridSearchCV(estimator=RandomForestClassifier(n_jobs=6, class_weight='balanced_subsample'), param_grid=params, scoring='f1_weighted')

In [33]:
fit_and_print(grid, X_test, y_test)

Best estimator:
 RandomForestClassifier(bootstrap=True, class_weight='balanced_subsample',
            criterion='gini', max_depth=None, max_features=3,
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=3, min_weight_fraction_leaf=0.0,
            n_estimators=5, n_jobs=6, oob_score=False, random_state=None,
            verbose=0, warm_start=False)
Accuracy score: 0.679829242263
Classification report
              precision    recall  f1-score   support

        Neg       0.07      0.08      0.07        77
    Neutral       0.76      0.86      0.81       707
        Pos       0.45      0.14      0.22       153

avg / total       0.65      0.68      0.65       937

Confusion matrix
 [[  6  70   1]
 [ 72 609  26]
 [  7 124  22]]


### Neural net

In [34]:
from sklearn.neural_network import MLPClassifier

In [35]:
params = {'alpha': [1e-5],
          'hidden_layer_sizes': [(4,3), (5,3), (8,3), (10,8,3), (10, 5, 3), (5,4,4,3)]}

grid = GridSearchCV(estimator=MLPClassifier(max_iter=2000, batch_size=128), param_grid=params, scoring='f1_weighted')

In [36]:
fit_and_print(grid, X_test, y_test)

C:\DEV\Anaconda3\envs\trading\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\DEV\Anaconda3\envs\trading\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Best estimator:
 MLPClassifier(activation='relu', alpha=1e-05, batch_size=128, beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(5, 3), learning_rate='constant',
       learning_rate_init=0.001, max_iter=2000, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)
Accuracy score: 0.595517609392
Classification report
              precision    recall  f1-score   support

        Neg       0.03      0.06      0.04        77
    Neutral       0.73      0.78      0.76       707
        Pos       0.22      0.01      0.02       153

avg / total       0.59      0.60      0.58       937

Confusion matrix
 [[  5  72   0]
 [149 551   7]
 [ 24 127   2]]
